In [4]:
import pandas as pd
import urllib.request
import urllib.parse
import json
from tqdm import tqdm_notebook

In [2]:
df = pd.read_excel('./data/물류창고정보_위치기반_231124.xls')
df = df.drop(['Unnamed: 5','Unnamed: 7'], axis=1)
col_name = list(df.iloc[1,:])
col_name[0:3] = df.columns[0:3]
df.columns = col_name
df = df.iloc[2:,].reset_index().drop(['index'], axis=1)
df = df.drop(['동수'], axis=1)

In [3]:
df

,상호명,성명,소재지,일반창고,냉동냉장,항만창고,보세창고,보관저장업,냉동냉장,축산물보관,냉동냉장
0,삼성전자로지텍,최수영,강원특별자치도 원주시 문막읍 반계리 1093번지 1호,4039,0,0,0,0,0,0,0
1,삼성전자로지텍㈜,최수영,세종특별자치시 부강면 갈산리 470번지 2호,8264.5,0,0,0,0,0,0,0
2,씨제이대한통운㈜ 남강원지사,강신호,강원특별자치도 원주시 흥업면 사제리 1339번지 1호,2430.08,0,0,0,0,0,0,0
3,주식회사 엘엑스판토스,최원혁,강원특별자치도 원주시 문막읍 동화리 608번지 42호,2953,0,0,0,0,0,0,0
4,씨제이대한통운 주식회사,강신호,강원특별자치도 원주시 호저면 주산리 275번지,2764,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
4899,어업회사법인주식회사해광,정선영,경상남도 창원시 마산회원구 봉암공단8길 5(봉암동),0,0,0,0,0,1032.13,0,0
4900,롯데글로벌로지스(주) 오포K-7 저온센터,박찬복,경기도 광주시 오포로 614,0,0,0,0,0,568.28,0,0
4901,(주)삼원상사,유이봉,경기도 용인시 처인구 백암면 석실로 28,0,0,0,0,0,408,0,0
4902,(주)세븐스타컴퍼니,김동진,경상북도 칠곡군 왜관읍 현대로 93,0,0,0,0,0,127.8,0,0


In [5]:
gb_df = df[df['소재지'].str.contains('경상북도|대구광역')]
gb_df = gb_df[~gb_df['소재지'].str.contains('울릉')].reset_index().drop(['index'], axis=1)
gb_df

,상호명,성명,소재지,일반창고,냉동냉장,항만창고,보세창고,보관저장업,냉동냉장,축산물보관,냉동냉장
0,경동물류(주),백영길,경상북도 칠곡군 연화리 700번지 영남종합물류 화물취급장 사동,3644,0,0,0,0,0,0,0
1,합동물류(주),백영창,경상북도 칠곡군 지천면 연화리 700번지 영남복합물류 사동,1797,0,0,0,0,0,0,0
2,씨제이대한통운(주),강신호,경상북도 칠곡군 가산면 다부리 445번지,3495,0,0,0,0,0,0,0
3,롯데하이마트(주),남창희,경상북도 칠곡군 왜관읍 금산리 390번지 외 1필지,11364,0,0,0,0,0,0,0
4,씨제이대한통운(주),강신호,대구광역시 달서구 장동 306번지 111호,2049.44,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
317,(주)한익스프레스경북센터,유인철 외 1명,경상북도 칠곡군 지천면 금호로1길 2-13,0,0,0,0,0,234,0,0
318,에이스후레쉬(주) 칠곡,전윤석,경상북도 칠곡군 지천면 산단터널로 240,0,0,0,0,0,1765,0,0
319,(주)한익스프레스칠곡센터,유인철 외 1명,경상북도 칠곡군 지천면 금호로 356,0,0,0,0,0,982.06,0,0
320,(주)포항국제물류센터 냉동창고,전성곤,경상북도 포항시 북구 흥해읍 영일만항로 236,0,0,0,0,0,6654.34,0,0


In [6]:
# 중복되는 칼럼명 변경
gb_df.columns = ['상호명', '성명', '소재지', '일반창고', '냉동냉장1', '항만창고', '보세창고', '보관저장업', '냉동냉장2',
       '축산물보관', '냉동냉장3']

In [9]:
# 냉동, 냉장 창고만 필터링
gb_df.iloc[:,3:] = gb_df.iloc[:,3:].astype(float)
gb_df = gb_df[(gb_df["냉동냉장1"] != 0) | (gb_df["냉동냉장2"] != 0) | (gb_df["냉동냉장3"] != 0) | (gb_df["축산물보관"] != 0)]
gb_df = gb_df.drop(['일반창고'], axis=1, inplace=False)
gb_df

,상호명,성명,소재지,냉동냉장1,항만창고,보세창고,보관저장업,냉동냉장2,축산물보관,냉동냉장3
9,대산농협,지용식,경상북도 김천시 대덕면 관기리 138-15,2626.24,0.0,0.0,0.0,0.0,0.0,0.0
10,김천유통영농조합법인,이기양,경상북도 김천시 대항면 이로리길 69,3427.13,0.0,0.0,0.0,0.0,0.0,0.0
11,김천유통영농조합법인,이기양,경상북도 김천시 다수동 510번지 1호,2159.59,0.0,0.0,0.0,0.0,0.0,0.0
19,한국농수산식품유통공사,김춘진,대구광역시 동구 괴전동 258번지,6634.0,0.0,0.0,0.0,0.0,0.0,0.0
20,씨제이대한통운(주),강신호,경상북도 경산시 압량면 부적리 151번지,3178.02,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
317,(주)한익스프레스경북센터,유인철 외 1명,경상북도 칠곡군 지천면 금호로1길 2-13,0.0,0.0,0.0,0.0,234.0,0.0,0.0
318,에이스후레쉬(주) 칠곡,전윤석,경상북도 칠곡군 지천면 산단터널로 240,0.0,0.0,0.0,0.0,1765.0,0.0,0.0
319,(주)한익스프레스칠곡센터,유인철 외 1명,경상북도 칠곡군 지천면 금호로 356,0.0,0.0,0.0,0.0,982.06,0.0,0.0
320,(주)포항국제물류센터 냉동창고,전성곤,경상북도 포항시 북구 흥해읍 영일만항로 236,0.0,0.0,0.0,0.0,6654.34,0.0,0.0


In [10]:
gb_df["면적"] = gb_df["냉동냉장1"] + gb_df['냉동냉장2'] + gb_df['냉동냉장3']
gb_df

,상호명,성명,소재지,냉동냉장1,항만창고,보세창고,보관저장업,냉동냉장2,축산물보관,냉동냉장3,면적
9,대산농협,지용식,경상북도 김천시 대덕면 관기리 138-15,2626.24,0.0,0.0,0.0,0.0,0.0,0.0,2626.24
10,김천유통영농조합법인,이기양,경상북도 김천시 대항면 이로리길 69,3427.13,0.0,0.0,0.0,0.0,0.0,0.0,3427.13
11,김천유통영농조합법인,이기양,경상북도 김천시 다수동 510번지 1호,2159.59,0.0,0.0,0.0,0.0,0.0,0.0,2159.59
19,한국농수산식품유통공사,김춘진,대구광역시 동구 괴전동 258번지,6634.0,0.0,0.0,0.0,0.0,0.0,0.0,6634.0
20,씨제이대한통운(주),강신호,경상북도 경산시 압량면 부적리 151번지,3178.02,0.0,0.0,0.0,0.0,0.0,0.0,3178.02
...,...,...,...,...,...,...,...,...,...,...,...
317,(주)한익스프레스경북센터,유인철 외 1명,경상북도 칠곡군 지천면 금호로1길 2-13,0.0,0.0,0.0,0.0,234.0,0.0,0.0,234.0
318,에이스후레쉬(주) 칠곡,전윤석,경상북도 칠곡군 지천면 산단터널로 240,0.0,0.0,0.0,0.0,1765.0,0.0,0.0,1765.0
319,(주)한익스프레스칠곡센터,유인철 외 1명,경상북도 칠곡군 지천면 금호로 356,0.0,0.0,0.0,0.0,982.06,0.0,0.0,982.06
320,(주)포항국제물류센터 냉동창고,전성곤,경상북도 포항시 북구 흥해읍 영일만항로 236,0.0,0.0,0.0,0.0,6654.34,0.0,0.0,6654.34


In [11]:
gb_df_width = gb_df[["상호명","소재지","면적"]]
gb_df_width

,상호명,소재지,면적
9,대산농협,경상북도 김천시 대덕면 관기리 138-15,2626.24
10,김천유통영농조합법인,경상북도 김천시 대항면 이로리길 69,3427.13
11,김천유통영농조합법인,경상북도 김천시 다수동 510번지 1호,2159.59
19,한국농수산식품유통공사,대구광역시 동구 괴전동 258번지,6634.0
20,씨제이대한통운(주),경상북도 경산시 압량면 부적리 151번지,3178.02
...,...,...,...
317,(주)한익스프레스경북센터,경상북도 칠곡군 지천면 금호로1길 2-13,234.0
318,에이스후레쉬(주) 칠곡,경상북도 칠곡군 지천면 산단터널로 240,1765.0
319,(주)한익스프레스칠곡센터,경상북도 칠곡군 지천면 금호로 356,982.06
320,(주)포항국제물류센터 냉동창고,경상북도 포항시 북구 흥해읍 영일만항로 236,6654.34


In [12]:
gb_df_coord = pd.read_csv("data/for_compare.csv")
gb_df_coord

,상호명,소재지,경도,위도
0,대산농협,경상북도 김천시 대덕면 관기리 138-15,127.974419,35.926329
1,김천유통영농조합법인,경상북도 김천시 대항면 이로리길 69,128.064241,36.126308
2,김천유통영농조합법인,경상북도 김천시 다수동 510번지 1호,128.062673,36.125406
3,한국농수산식품유통공사,대구광역시 동구 괴전동 258번지,128.740345,35.870344
4,동원로엑스(주),경상북도 경산시 진량읍 신상리 1187번지,128.834546,35.877169
...,...,...,...,...
161,(주)한익스프레스경북센터,경상북도 칠곡군 지천면 금호로1길 2-13,128.444991,35.956688
162,에이스후레쉬(주) 칠곡,경상북도 칠곡군 지천면 산단터널로 240,128.442744,35.954255
163,(주)한익스프레스칠곡센터,경상북도 칠곡군 지천면 금호로 356,128.449856,35.961823
164,(주)포항국제물류센터 냉동창고,경상북도 포항시 북구 흥해읍 영일만항로 236,129.434048,36.100115


In [25]:
merged_df = pd.merge(gb_df_width, gb_df_coord, on=['상호명', '소재지'], how='right')
merged_df = merged_df.drop_duplicates(keep='last')
merged_df

,상호명,소재지,면적,경도,위도
0,대산농협,경상북도 김천시 대덕면 관기리 138-15,2626.24,127.974419,35.926329
1,김천유통영농조합법인,경상북도 김천시 대항면 이로리길 69,3427.13,128.064241,36.126308
2,김천유통영농조합법인,경상북도 김천시 다수동 510번지 1호,2159.59,128.062673,36.125406
3,한국농수산식품유통공사,대구광역시 동구 괴전동 258번지,6634.0,128.740345,35.870344
4,동원로엑스(주),경상북도 경산시 진량읍 신상리 1187번지,1854.6,128.834546,35.877169
...,...,...,...,...,...
185,(주)한익스프레스경북센터,경상북도 칠곡군 지천면 금호로1길 2-13,234.0,128.444991,35.956688
186,에이스후레쉬(주) 칠곡,경상북도 칠곡군 지천면 산단터널로 240,1765.0,128.442744,35.954255
187,(주)한익스프레스칠곡센터,경상북도 칠곡군 지천면 금호로 356,982.06,128.449856,35.961823
188,(주)포항국제물류센터 냉동창고,경상북도 포항시 북구 흥해읍 영일만항로 236,6654.34,129.434048,36.100115
